# Teste da API Comtrade

Info: https://comtradedeveloper.un.org

Specs: https://comtradedeveloper.un.org/api-details#api=comtrade-v1&operation=get-get



In [25]:
# %pip install requests

Note: you may need to restart the kernel to use updated packages.


In [26]:
# %pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [27]:
# %pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


Ler os códigos HS

Obter em https://github.com/datasets/harmonized-system/blob/master/data/harmonized-system.csv

Copiar para directoria `support`


In [28]:
import pandas as pd

hs_codes_df = pd.read_csv('downloads/support/harmonized-system.csv')

Fazer um dicionário para os códigos de nível 2

In [29]:
hs_codes_l2 = hs_codes_df[hs_codes_df.level == 2]
hs_codes_l2[['hscode','description']]
hs_l2_map = dict(zip(hs_codes_l2.hscode, hs_codes_l2.description))


Aqui colocar os parâmetros gerais que não mudam.

In [30]:
baseUrl = "https://comtradeapi.un.org/public/v1"

m49_codes_cn_plp ={
    "024":"Angola",
    "076":"Brazil",
    "132":"Cabo Verde",
    "156":"China",
    "344":"China, Hong Kong",
    "446":"China, Macau",
    "226":"Guiné Equatorial",
    "624":"Guiné-Bissau",
    "508":"Moçambique",
    "620":"Portugal",
    "678":"São Tomé e Príncipe",
    "626":"Timor-Leste"
}
m49_china = "156"
m49_hong_kong = "344"
m49_macau = "446"
m49_plp = set(m49_codes_cn_plp.keys())-{m49_china,m49_hong_kong,m49_macau}
m49_plp_list = ",".join(m49_plp)

Aqui os parâmetros de cada chamada à API.

In [49]:
typeCode = "C" # C for commodities, S for Services
freqCode = "A" # A for annual and M for monthly
clCode = "HS" #Trade classifications Available values : HS, SS, B4, B5, EB, EB10, EB02, EBSDMX
reporterCode = m49_china # China, HK 344, Macau 446, see https://unstats.un.org/unsd/methodology/m49/
partnerCode = m49_plp_list
period="2020"
cmdCode="AG2" # TOTAL, AG2 (HS level 2)
flowCode = "M" # M=import, X=export,outros: RX, RM, MIP, XIP, MOP, XOP, MIF, XIF, DX, FM


In [50]:
import requests
requestUrl=f"{baseUrl}/preview/{typeCode}/{freqCode}/{clCode}"
resp = requests.get(requestUrl,
        {
        'reporterCode':m49_china,
        'period':period,
        'partnerCode':m49_plp_list,
        'cmdCode':cmdCode,
        'flowCode':flowCode
        },
        timeout=10)

In [51]:
import json
import pandas as pd

results = json.loads(resp.content)['data']
df = pd.DataFrame(results)
# convert result country codes to three-digit codes
df.reporterCode = df.reporterCode.astype('str').str.zfill(3)
df.partnerCode = df.partnerCode.astype('str').str.zfill(3)
# print(df.columns.values)

In [58]:


df.reporterDesc = df.reporterCode.map(m49_codes_cn_plp)
df.partnerDesc = df.partnerCode.map(m49_codes_cn_plp)
df.cmdDesc = df.cmdCode.map(hs_l2_map)

df['primaryValueFormated'] = df.primaryValue.map('{:,}'.format)
cols = ['typeCode','freqCode','reporterDesc','partnerDesc','refYear','cmdCode','cmdDesc','flowCode','primaryValueFormated','primaryValue']

pco = df[cols].sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] < 6]
pco_top5

,typeCode,freqCode,reporterDesc,partnerDesc,refYear,cmdCode,cmdDesc,flowCode,primaryValueFormated,primaryValue,rank
4,C,A,China,Angola,2020,27,"Mineral fuels, mineral oils and products of th...",M,"14,686,950,196.0",1.468695e+10,1.0
2,C,A,China,Angola,2020,25,"Salt; sulphur; earths, stone; plastering mater...",M,"29,428,961.0",2.942896e+07,2.0
14,C,A,China,Angola,2020,71,"Natural, cultured pearls; precious, semi-preci...",M,"20,014,740.0",2.001474e+07,3.0
16,C,A,China,Angola,2020,74,Copper and articles thereof,M,"9,938,185.0",9.938185e+06,4.0
3,C,A,China,Angola,2020,26,"Ores, slag and ash",M,"6,763,117.0",6.763117e+06,5.0
46,C,A,China,Brazil,2020,26,"Ores, slag and ash",M,"27,701,000,048.0",2.770100e+10,1.0
33,C,A,China,Brazil,2020,12,Oil seeds and oleaginous fruits; miscellaneous...,M,"24,906,157,697.0",2.490616e+10,2.0
47,C,A,China,Brazil,2020,27,"Mineral fuels, mineral oils and products of th...",M,"14,303,779,665.0",1.430378e+10,3.0
23,C,A,China,Brazil,2020,02,Meat and edible meat offal,M,"7,426,321,503.0",7.426322e+09,4.0
67,C,A,China,Brazil,2020,47,Pulp of wood or other fibrous cellulosic mater...,M,"3,661,957,178.0",3.661957e+09,5.0


In [54]:
df[cols].to_excel(f"./downloads/dados_comtrade_{period}_{cmdCode}.xlsx")
pco.to_excel(f"./downloads/paises_cmd_ordenados_{period}.xlsx")
pco_top5.to_excel(f"./downloads/paises_cmd_top5_{period}.xlsx")